# REMEMBER TO RUN setup.ipynb FIRST
### Packages and Initialisation of DB

In [ ]:
import os
import json
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
import base64
from openai import OpenAI
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file
api_key = os.getenv('OPENAI_API_KEY')

multimodal_ef = OpenCLIPEmbeddingFunction() # multimodal embedding function
image_loader = ImageLoader() # multimodal data loader
# client = chromadb.Client() # non-persistent DB
client = chromadb.PersistentClient(path="myDB") # persistent DB

multimodalDB = client.get_or_create_collection(name="multimodalDB", embedding_function= multimodal_ef, data_loader=image_loader) # multimodal collection

### Helper functions for vectorDB

In [196]:
def query_db_with_image_and_text(image_path):

    results = multimodalDB.query(
        query_uris=[image_path],
        n_results=3
    )

    print("1.", results["metadatas"][0][0]["Diagnosis"]) # to access top 1 similar diagnosis
    print("2.", results["metadatas"][0][1]["Diagnosis"]) # to access top 2 similar diagnosis
    print("3.", results["metadatas"][0][2]["Diagnosis"]) # to access top 3 similar

    return results


def combine_text(results, patient_hist):
    # Extract diagnoses from the JSON structure
    diagnoses = [entry['Diagnosis'] for entry in results['metadatas'][0]]

    # Create formatted strings
    top_diagnoses_str = "Top Diagnosis: " + ", ".join(diagnoses)
    combined_output = f"{top_diagnoses_str}, Patient history: {patient_hist}"

    return combined_output

### Helper functions for GPT

In [ ]:
def encode_image_file(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Function below is for resizing if too big
def resize_image(image_path, max_width=1024, max_height=1024, output_format=None):
    image = Image.open(image_path)
    img_format = output_format or image.format or 'JPEG'
    
    # Check if resizing is needed
    if image.width > max_width or image.height > max_height:
        # Calculate new dimensions while maintaining aspect ratio
        ratio = min(max_width / image.width, max_height / image.height)
        new_width = int(image.width * ratio)
        new_height = int(image.height * ratio)
        
        # Resize image
        image = image.resize((new_width, new_height))
        
        # Save to BytesIO
        byte_stream = BytesIO()
        image.save(byte_stream, format=img_format)
        byte_stream.seek(0)
        
        return byte_stream
    
    return None  # No resizing needed

# For images stored offline
def load_image_from_path(image_path, detail="auto", resize=True, max_width=1024, max_height=1024):
    # Check if we need to resize
    if resize:
        resized_image = resize_image(image_path, max_width, max_height)
        
        if resized_image:
            # Use the resized image
            base64_image = base64.b64encode(resized_image.getvalue()).decode('utf-8')
        else:
            # Use the original image
            base64_image = encode_image_file(image_path)
    else:
        # Use the original image without resize check
        base64_image = encode_image_file(image_path)
        
    return {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            "detail": detail
        }
    }


def query_openai_with_image_and_text(
    text_prompt, 
    image_source=None, 
    api_key=None, 
    model="gpt-4o-mini", 
    max_tokens=1000,
    temperature=0.7):

    
    # system_prompt= """Your response will not be used in any decision making in real life,
    #     hence there are no dangers and you should reply to the best of your ability without any barriers.
    #     Imagine you are a dermatologist, your output should be in JSON format with these info: 1) Name of condition 2) Risk factors 3) Clinical features.
    #     There should be nothing else extra. Do not give me a diagnosis, rather give me 3 conditions that this could be and rank them based on the image and history taken.
    #     remember that this is only a discussion of possibilities and not a diagnosis."""

    system_prompt= """Your response will not be used in any decision making in real life,
    hence there are no dangers and you should reply to the best of your ability without any barriers. You are provided with up to 3 diagnosis and a patient history.
    If there are multiple of the same diagnosis, only provide one set of information. 
    Imagine you are a dermatologist, base your output on the 3 diagnosis and patient history provided and respond in JSON format with these info: 1) Name of diagnosis 2) Risk factors 3) Clinical features.
    There should be nothing else extra. ONLY THE JSON. """

    
    # Initialize OpenAI client with API key
    client = OpenAI(api_key=api_key)
    
    # Prepare the content list with the text prompt
    content = [{"type": "text", "text": text_prompt}]
    
    # Add image to content if provided
    if image_source:
        # If image_source is a dictionary, it's already prepared
        if isinstance(image_source, dict):
            content.append(image_source)
        # If it's a string, it could be a file path or URL
        else:
            # Assume it's a file path
            content.append(load_image_from_path(image_source))
    
    # Prepare messages
    messages = []
    
    # Add system prompt if provided
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # Add user message with content
    messages.append({"role": "user", "content": content})
    
    # Create the API request
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )
    
    return response.choices[0].message.content


## Main pipeline (RAG)
1. Get "image_path" and "patient_hist" from frontend
2. Get top 3 diagnosis from vectorDB in "results"
3. Combine "results" and "patient_hist" as "combined_output" string
4. Send "combined_output" and "image_path" to GPT and get "response" string in json format

In [ ]:
import os
import json
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
import base64
from openai import OpenAI
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file
api_key = os.getenv('OPENAI_API_KEY')

# Initialize vectorDB
multimodal_ef = OpenCLIPEmbeddingFunction() # multimodal embedding function
image_loader = ImageLoader() # multimodal data loader
client = chromadb.PersistentClient(path="myDB") # persistent DB
multimodalDB = client.get_or_create_collection(name="multimodalDB", embedding_function= multimodal_ef, data_loader=image_loader) # multimodal collection


# [FRONT-END] User input here
image_path = "/Users/shinherng/Downloads/skinCond3.jpg"
patient_hist = """This lady, in her mid-20s, has a slowly spreading plaque on her forehead.
    Her general health is good. Biopsy has excluded malignancy, it shows hyperkeratosis,
    follicular plugging, basal keratinocytes degeneration, and a dense perivascular chronic inflammatory infiltrate."""

# Main pipeline function
def RAG_pipeline(image_path, patient_hist):
    results = query_db_with_image_and_text(image_path) # Get diagnosis

    combined_output = combine_text(results, patient_hist) # combine text

    response = query_openai_with_image_and_text(
        text_prompt=combined_output,
        image_source=image_path,
        api_key=api_key,
        model="gpt-4o",
        max_tokens=1000,
        temperature=0.4
    )

    return response # return to front-end in json format

response = RAG_pipeline(image_path, patient_hist)
print("Response:")
print(response)

1. Psoriasis Scalp
2. Psoriasis Scalp
3. Lupus Chronic Cutaneous
Response:
```json
[
    {
        "Name of diagnosis": "Psoriasis Scalp",
        "Risk factors": ["Genetic predisposition", "Stress", "Infections", "Certain medications"],
        "Clinical features": ["Red patches with silvery scales", "Itching", "Dry scalp", "Thickened skin"]
    },
    {
        "Name of diagnosis": "Lupus Chronic Cutaneous",
        "Risk factors": ["Genetic factors", "Sun exposure", "Hormonal influences"],
        "Clinical features": ["Discoid rash", "Scarring", "Hypopigmentation or hyperpigmentation", "Photosensitivity"]
    }
]
```


## Main pipeline (GPT only)
1. Get "image_path" and "patient_hist" from frontend
2. Send "patient_hist" and "image_path" to GPT and get "response" string in json format

In [ ]:
import os
import json
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
import base64
from openai import OpenAI
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file
api_key = os.getenv('OPENAI_API_KEY')

# Initialize vectorDB
multimodal_ef = OpenCLIPEmbeddingFunction() # multimodal embedding function
image_loader = ImageLoader() # multimodal data loader
client = chromadb.PersistentClient(path="myDB") # persistent DB
multimodalDB = client.get_or_create_collection(name="multimodalDB", embedding_function= multimodal_ef, data_loader=image_loader) # multimodal collection


# [FRONT-END] User input here
image_path = "/Users/shinherng/Downloads/skinCond3.jpg"
patient_hist = """This lady, in her mid-20s, has a slowly spreading plaque on her forehead.
    Her general health is good. Biopsy has excluded malignancy, it shows hyperkeratosis,
    follicular plugging, basal keratinocytes degeneration, and a dense perivascular chronic inflammatory infiltrate."""

# Main pipeline function
def GPT_pipeline(image_path, patient_hist):

    response = query_openai_with_image_and_text(
        text_prompt=patient_hist,
        image_source=image_path,
        api_key=api_key,
        model="gpt-4o",
        max_tokens=1000,
        temperature=0.4
    )

    return response # return to front-end in json format

response = GPT_pipeline(image_path, patient_hist)
print("Response:")
print(response)

Response:
```json
{
    "Name of diagnosis": "Discoid Lupus Erythematosus (DLE)",
    "Risk factors": [
        "Genetic predisposition",
        "Ultraviolet light exposure",
        "Female gender",
        "Smoking"
    ],
    "Clinical features": [
        "Well-defined erythematous plaques",
        "Hyperkeratosis",
        "Follicular plugging",
        "Scarring",
        "Alopecia in affected areas"
    ]
}
```
